# Домашнее задание №1

### Пункт 1. Загрузка данных из xlsx файлов в Python и в базу данных SQL

#### В ячейку ниже впишите в двойных кавычках (" "):
* в переменную **_my__path_** путь вашей папки, в которой расположены папки *applications* и *contracts* (в путь включается название самой папки, в текущем примере папка со всеми данными называется IT)
* в переменную **_mydatabase_** название созданной вами базы данных
* в переменную **_mypassword_** ваш пользовательский пароль для работы с MySQL
* в переменную **_myuser_** ваше имя пользователя MySQL (по умолчанию *root*)

In [1]:
my_path = "C:/Users/plaps/Desktop/IT"
mydatabase = "loan"
mypassword = "MySQL2018"
myuser = "root"

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import datetime
from dateutil.parser import parse
from IPython.display import display, HTML
from sklearn import preprocessing
import mysql
import mysql.connector
from mysql.connector import Error

In [3]:
# создаем список файлов в каждой папке
applications = [f for f in listdir(my_path+'/applications') if isfile(join(my_path+'/applications', f))] 
contracts = [f for f in listdir(my_path+'/contracts') if isfile(join(my_path+'/contracts', f))]

In [4]:
def initial_table(file_name, folder_type):
    #преобразует исходную таблицу в зависимости от того, по заявкам она или по контрактам в вид, удобный для дальнейшей работы
    if folder_type == 'applications':
        df1 = pd.read_excel(my_path+"/"+folder_type+"/"+file_name).reset_index()
        df1.columns = range(df1.shape[1])
        df1 = df1.drop([9,10,11]).reset_index(drop = True)
    else:
        df1 = pd.read_excel(my_path+"/"+folder_type+"/"+file_name)
        df1.columns = range(df1.shape[1])
    if df1.shape[0] == 16:
        df1 = df1.append(pd.Series(np.full(df1.shape[1], np.nan), index = df1.columns), ignore_index = True)
    return df1

In [5]:
def new_col_names_and_data_col_index (df1):
    #из таблицы собирается полный набор наименований признаков на базе предпосылки о том, что такие наименования встречаются
    #через строчку (итерируемся по списку строк с шагом 2)
    #для всех найденных имен собираются их индексы в первоначальной анкете (по столбцам, для каждой строки из списка)
    #creating list of columns names 
    row_ind = range(1,df1.shape[0],2)
    col_names = [] 
    col_ind = []
    for i in row_ind:
        ls = df1.iloc[i].tolist()
        ls_clean = [x for x in ls if str(x) !='nan' and str(x) != 'NaT' ]
        col_names.extend(ls_clean)
        this_row_col_ind = []
        for j in ls_clean:
            this_row_col_ind.append(ls.index(j))
        col_ind.append(this_row_col_ind)
    col_names.append('Date')
    
    # renaming duplicates with progressing numbers
    from collections import Counter
    counts = Counter(col_names)
    for s, num in counts.items():
        if num > 1:
            for i in range (1,num +1):
                            col_names[col_names.index(s)] = s + str(i)
    return col_names, col_ind

In [6]:
#отдельно создаём под каждый тип данных (заявки и контракты) пустой датафрейм, к которому с помощью функции внизу добавляем строки
appl_inf = pd.DataFrame(columns = new_col_names_and_data_col_index(initial_table(applications[0],'applications'))[0])
contr_inf = pd.DataFrame(columns = new_col_names_and_data_col_index(initial_table(contracts[0],'contracts'))[0])

def new_rows_append (df0,df1, inf):
    row_ind = range(1,df0.shape[0],2)
    col_names,col_ind = new_col_names_and_data_col_index(df0)
    dates_columns = []
    for i in col_names:
        if "Date" in i:
            dates_columns.append(col_names.index(i))
    new_row = []
    for i in range(0,len(row_ind)):
        row_n = df1.iloc[row_ind[i]+1][col_ind[i]]
        new_row.extend(row_n)  
    # добавляем в конце дату формы,т.к. её расположение отличается от стандартного расположения остальных данных.  
    new_row.append(df1.iloc[0][df1.shape[1]-1])
    
    # Во избежании ошибок с форматом дат, преобразуем также все данные из строк с датами в формате строк в даты, 
    # проверяем год на лимит питона
    for i in range(0,len(dates_columns)):
        if type(new_row[dates_columns[i]]) == str:
            new_row[dates_columns[i]] = datetime.strptime(new_row[dates_columns[i]], '%m.%d.%Y')
        if new_row[dates_columns[i]].year > 2300:
            new_row[dates_columns[i]] = float('nan')
            
    inf = inf.append(pd.Series(new_row, index = inf.columns), ignore_index = True)
    return inf

#### Получаем таблицу с заявками

In [7]:
for i in applications:
    df1 = initial_table(i,'applications')
    df0 = initial_table(applications[0],'applications')
    appl_inf = new_rows_append (df0,df1, appl_inf)
cols_appl = list (appl_inf)
cols_appl.insert(0, cols_appl.pop(cols_appl.index('Identity Number')))
appl_inf = appl_inf.loc[:,cols_appl]
appl_inf = appl_inf.where((pd.notnull(appl_inf)), None)
appl_inf_no_nan = appl_inf.dropna(axis=1, how='all')
appl_inf_no_nan = appl_inf_no_nan.rename(index=str, columns={"Date": "Application_date"})

#### Удалим необязательные поля *Name* и *Date appointed*, посмотрим, как выглядит таблица, которая отправится в базу данных

In [8]:
print(list(appl_inf_no_nan)) #посмотрим на оставшиеся поля и удалим ненужные 

['Identity Number', 'Name', 'Date of Birth', 'Gender', 'Employed By', 'Date appointed', 'Issue Date', 'Education', 'Children', 'Family', 'Marital Status', 'Position', 'Income', 'Income Type', 'Housing', 'House ownership', 'Age of Car (if owned)', 'Application_date']


In [9]:
appl_inf_no_nan = appl_inf_no_nan.drop(['Name', 'Date appointed'], axis = 1)
appl_inf_no_nan.head()

,Identity Number,Date of Birth,Gender,Employed By,Issue Date,Education,Children,Family,Marital Status,Position,Income,Income Type,Housing,House ownership,Age of Car (if owned),Application_date
0,100003,1968-07-22,Female,School,2013-08-29,Higher education,0,2,Married,Core staff,270000,State servant,House / apartment,N,None,2014-06-16
1,100004,1964-10-11,Male,Government,2010-01-28,Secondary / secondary special,0,1,Single / not married,Laborers,67500,Working,House / apartment,Y,26,2017-02-01
2,100006,1964-08-29,Female,Business Entity Type 3,2010-08-01,Secondary / secondary special,0,2,Civil marriage,Laborers,135000,Working,House / apartment,Y,None,2016-10-09
3,100007,1963-09-02,Male,Religion,2008-03-18,Secondary / secondary special,0,1,Single / not married,Core staff,121500,Working,House / apartment,Y,None,2017-05-09
4,100008,1968-10-27,Male,Other,2013-11-24,Secondary / secondary special,0,2,Married,Laborers,99000,State servant,House / apartment,Y,None,2015-03-16


#### Получаем таблицу с контрактами

In [10]:
for i in contracts:
    df1 = initial_table(i,'contracts')
    df0 = initial_table(contracts[0],'contracts')
    contr_inf = new_rows_append (df0,df1, contr_inf)
cols_contr = list (contr_inf)
cols_contr.insert(0, cols_contr.pop(cols_contr.index('Identity Number')))
contr_inf = contr_inf.loc[:,cols_contr]
contr_inf = contr_inf.where((pd.notnull(contr_inf)), None)
contr_inf_no_nan = contr_inf.dropna(axis=1, how='all')
contr_inf_no_nan = contr_inf_no_nan.rename(index=str, columns={"Date": "Contract_date"})

#### Удалим необязательное поле *Borrower*, посмотрим, как выглядит таблица, которая отправится в базу данных

In [11]:
print(list(contr_inf_no_nan))

['Identity Number', 'Borrower', 'Contract Number', 'Amount', 'Type', 'Term (month)', 'Annuity', 'Contract_date']


In [12]:
contr_inf_no_nan = contr_inf_no_nan.drop(['Borrower'], axis = 1)
contr_inf_no_nan.head()

,Identity Number,Contract Number,Amount,Type,Term (month),Annuity,Contract_date
0,100009,100022,1560726,Cash loans,41,41301,2015-02-11
1,100047,100052,1193580,Cash loans,37,35028,2016-08-30
2,100025,100068,1.13257e+06,Cash loans,33,37561.5,2018-12-30
3,100064,100113,298728,Cash loans,22,15381,2018-02-19
4,100023,100122,544491,Cash loans,34,17563.5,2016-07-31


#### Создание и заполнение таблиц в базе данных SQL

In [13]:
# подключаемся к базе данных
database = mysql.connector.connect(user=myuser, password=mypassword,
                              host='localhost',
                              database=mydatabase,auth_plugin='mysql_native_password')
mycursor = database.cursor(buffered=True)

In [14]:
# создаем таблицу заявок (заданный порядок переменных важен, менять нельзя)
mycursor.execute(
"""CREATE TABLE applications (id_number INT,date_of_birth DATE, gender VARCHAR(10), employed_by VARCHAR(50), issue_date DATE, 
education VARCHAR (50), children INT, family INT, marital_status VARCHAR (50), position VARCHAR(50), income INT, 
income_type VARCHAR(50), housing VARCHAR (50), house_ownership VARCHAR (50), age_of_car INT, application_date DATE, 
PRIMARY KEY (id_number)  )""")

In [15]:
# заполняем значениями таблицу applications
sql = "INSERT INTO applications VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for i in range(0, appl_inf_no_nan.shape[0]):
    val = tuple(appl_inf_no_nan.iloc[i])
    mycursor.execute(sql, val)

    database.commit()

In [16]:
# создаем таблицу контрактов (заданный порядок переменных важен, менять нельзя)
mycursor.execute(
"""CREATE TABLE contracts (id_number INT, contract_number INT, amount INT, type VARCHAR(50), term_month INT, annuity INT, 
contract_date DATE, PRIMARY KEY (id_number)  )""")

In [17]:
# заполняем значениями таблицу contracts
sql = "INSERT INTO contracts VALUES (%s, %s, %s, %s, %s, %s, %s)"
for i in range(0, contr_inf_no_nan.shape[0]):
    val = tuple(contr_inf_no_nan.iloc[i])
    mycursor.execute(sql, val)

    database.commit()

### Пункт 2. Проверка на корректность значений, исправление некоторых ошибок

In [18]:
# позволяет посмотреть для всех текстовых переменных частоту появлений каждой категории,
# для каждой переменной формата integer или дата посмотреть на максимальное, минимальное значение и количество пропусков
def value_check (table, data_type):
    mycursor.execute("""SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
    WHERE DATA_TYPE = '{0}' AND TABLE_SCHEMA = '{1}'
    AND TABLE_NAME = '{2}'""".format(data_type, mydatabase, table))
    data_type_col = pd.DataFrame(mycursor.fetchall())[0].tolist()
    for i in range(0,len(data_type_col)):
        if data_type == 'varchar':
            mycursor.execute("""SELECT {0}, COUNT({0}) AS count FROM {1} 
            WHERE {0} IS NOT NULL GROUP BY {0} UNION ALL SELECT {0}, 
            COUNT(CASE WHEN {0} IS NULL THEN 1 END) AS count FROM {1} 
            WHERE {0} IS NULL GROUP BY {0}""".format(data_type_col[i], table))
        if data_type == 'int' or data_type == 'date':
            mycursor.execute("""SELECT MAX({0}), MIN({0}), 
            COUNT(CASE WHEN {0} IS NULL THEN 1 END) AS null_count FROM {1}""".format(data_type_col[i], table))
        data = pd.DataFrame(mycursor.fetchall())
        data.columns = [x[0] for x in mycursor.description]
        print ("Проверка обозначений по полю "+data_type_col[i])
        display(data)
        print("- "*30)
        print()

In [19]:
# проверяет, есть ли в числовых данных отрицательные значения (в нашем случае для всех полей такое значение некорректно) 
# и, если есть, заменяет все отрицательные числовые значения на NULL
def int_negative_value_update (table):
    mycursor.execute("""SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
    WHERE DATA_TYPE = 'int' AND TABLE_SCHEMA = '{0}'
    AND TABLE_NAME = '{1}'""".format(mydatabase, table))
    data_type_col = pd.DataFrame(mycursor.fetchall())[0].tolist()
    for i in range(0,len(data_type_col)):
        mycursor.execute("""SELECT COUNT(CASE WHEN {0}<0 THEN 1 END) FROM {1}""".format(data_type_col[i], table)) 
        data = mycursor.fetchall()[0][0]
        if data != 0:
            mycursor.execute("""UPDATE {0} SET {1}= NULL WHERE {1}<0""".format(table, data_type_col[i]))
            database.commit()
            print ("Отрицательные значения найдены для поля "+ data_type_col[i] + ", некорректные значения заменены на None")
        else:
            print ("Отрицательные значения для поля "+ data_type_col[i] + " не найдены, ошибок нет")

In [20]:
# поиск и исправление ошибок в датах
def date_value_update (table):
    mycursor.execute("""SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
    WHERE DATA_TYPE = 'date' AND TABLE_SCHEMA = '{0}'
    AND TABLE_NAME = '{1}'""".format(mydatabase, table))
    data_type_col = pd.DataFrame(mycursor.fetchall())[0].tolist()
    #проверка на даты из будущего
    for i in range(0,len(data_type_col)):
        mycursor.execute("""SELECT COUNT(CASE WHEN {0}>'2018-12-31' THEN 1 END) FROM {1}""".format(data_type_col[i], table)) 
        data = mycursor.fetchall()[0][0]
        if data != 0:
            mycursor.execute("""UPDATE {0} SET {1}= NULL WHERE {1}>'2018-12-31'""".format(table, data_type_col[i]))
            database.commit()
            print ("Среди дат по полю "+ data_type_col[i] + " найдены будущие значения, некорректные значения заменены на None")
        else:
            print ("Среди дат по полю "+ data_type_col[i] + " некорректные значения не найдены")
    print("- "*30)
    if table == 'applications':
        #создание поля возраста
        mycursor.execute("""ALTER TABLE applications ADD age INT""")
        database.commit()
        mycursor.execute("""UPDATE applications SET age = IF (MONTH(CURRENT_TIMESTAMP) <> MONTH(date_of_birth),
        YEAR(CURDATE())- YEAR(date_of_birth) - (MONTH(CURRENT_TIMESTAMP) < MONTH(date_of_birth)), 
        YEAR(CURDATE())- YEAR(date_of_birth) - (DAY(CURRENT_TIMESTAMP) < DAY(date_of_birth))) WHERE date_of_birth IS NOT NULL""")
        database.commit()
        mycursor.execute("""ALTER TABLE applications DROP date_of_birth""")
        database.commit()
        #проверка возраста до 18 лет
        mycursor.execute("""SELECT id_number, age FROM applications WHERE age < 18""")
        data = mycursor.fetchall()
        if not data:
            print("Заёмщиков младше 18 лет не найдено")
        else:
            data1 = pd.DataFrame(data)
            data1.columns = [x[0] for x in mycursor.description]
            print("Найдены следующие заёмщики младше 18 лет:")
            display(data1)

#### Проверка значений в таблице applications

In [21]:
table = 'applications'

Проверим таблицу заявок по всем категориальным полям

In [22]:
data_type = 'varchar'
value_check(table, data_type)

Проверка обозначений по полю education


,education,count
0,Higher education,18
1,Secondary / secondary special,44
2,Incomplete higher,1
3,None,1


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю employed_by


,employed_by,count
0,School,2
1,Government,3
2,Business Entity Type 3,12
3,Religion,1
4,Other,3
5,XNA,9
6,Electricity,1
7,Medicine,2
8,Business Entity Type 2,3
9,Self-employed,8


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю gender


...

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю marital_status


,marital_status,count
0,Married,39
1,Single / not married,12
2,Civil marriage,8
3,Widow,3
4,Separated,1
5,None,1


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю position


,position,count
0,Core staff,7
1,Laborers,17
2,Accountants,2
3,Managers,6
4,<undefined>,14
5,Drivers,6
6,Sales staff,6
7,Cleaning staff,1
8,Cooking staff,1
9,Private service staff,2


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



Как мы видим, нарушения в обозначениях не наблюдаются нигде, кроме поля position, поскольку в нём присутствует значение undefined. Заменим данное значение в таблице на стандартное для пропуска None.

In [23]:
mycursor.execute("UPDATE applications SET position = NULL WHERE position = '<undefined>'")
mycursor.execute("""SELECT ROW_COUNT()""")
counter = mycursor.fetchall()[0][0]
database.commit()
print(counter, " значений <undefined> заменены на None")

14  значений <undefined> заменены на None


Проверка на соотношение количества членов семьи и детей (количество членов семьи не должно превышать сумму детей и родителей в зависимости от семейного положения родителей)

In [24]:
mycursor.execute("""SELECT id_number, family, children, marital_status FROM applications 
WHERE marital_status = 'Married' OR marital_status = 'Civil marriage' GROUP BY id_number
HAVING COUNT(CASE WHEN family <> children + 2 THEN 1 END) UNION ALL SELECT id_number, family, children, marital_status FROM applications 
WHERE marital_status = 'Separated' OR marital_status = 'Widow' OR marital_status = 'Single / not married' GROUP BY id_number
HAVING COUNT(CASE WHEN family <> children + 1 THEN 1 END)""")
data = mycursor.fetchall()
if not data:
    print('Ошибки в соотношении количества членов семьи и детей не найдены')
else:
    data1 = pd.DataFrame(data)
    data1.columns = [x[0] for x in mycursor.description]
    print('Ошибки в соотношении количества членов семьи и детей найдены по следующим заёмщикам:')
    display(data1)

Ошибки в соотношении количества членов семьи и детей не найдены


Проверка числовых переменных:

In [25]:
data_type = 'int'
value_check(table, data_type)

Проверка обозначений по полю age_of_car


,MAX(age_of_car),MIN(age_of_car),null_count
0,26,1,46


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю children


,MAX(children),MIN(children),null_count
0,2,0,1


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю family


...

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю id_number


,MAX(id_number),MIN(id_number),null_count
0,100076,100003,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю income


,MAX(income),MIN(income),null_count
0,540000,38419,1


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



Если в данных есть отрицательные значения, исправим это, если нет - укажем, что ошибка не обнаружена

In [26]:
int_negative_value_update('applications')

Отрицательные значения для поля age_of_car не найдены, ошибок нет
Отрицательные значения для поля children не найдены, ошибок нет
Отрицательные значения для поля family не найдены, ошибок нет
Отрицательные значения для поля id_number не найдены, ошибок нет
Отрицательные значения для поля income не найдены, ошибок нет


Проверим значения в полях с датами:

In [27]:
data_type = 'date'
value_check(table, data_type)

Проверка обозначений по полю application_date


,MAX(application_date),MIN(application_date),null_count
0,2018-12-29,2014-05-06,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю date_of_birth


,MAX(date_of_birth),MIN(date_of_birth),null_count
0,1997-12-03,1947-10-14,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю issue_date


,MAX(issue_date),MIN(issue_date),null_count
0,2017-03-28,2001-10-01,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



Проверим, есть ли в данных с датами некорректные значения (даты из будущего, займщики младше 18 лет), создадим поле возраста

In [28]:
date_value_update(table)

Среди дат по полю application_date некорректные значения не найдены
Среди дат по полю date_of_birth некорректные значения не найдены
Среди дат по полю issue_date некорректные значения не найдены
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Заёмщиков младше 18 лет не найдено


#### Проверка значений в таблице контрактов

In [29]:
table = 'contracts'

Проверим таблицу контрактов по всем категориальным полям

In [30]:
data_type = 'varchar'
value_check(table, data_type)

Проверка обозначений по полю type


,type,count
0,Cash loans,52
1,Revolving loans,9


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



Как мы видим, нарушения в обозначениях не наблюдаются

Проверка числовых переменных:

In [31]:
data_type = 'int'
value_check(table, data_type)

Проверка обозначений по полю amount


,MAX(amount),MIN(amount),null_count
0,1663988,80865,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю annuity


,MAX(annuity),MIN(annuity),null_count
0,86990,5882,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю contract_number


...

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю id_number


,MAX(id_number),MIN(id_number),null_count
0,100075,100002,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Проверка обозначений по полю term_month


,MAX(term_month),MIN(term_month),null_count
0,41,12,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



In [32]:
int_negative_value_update(table)

Отрицательные значения для поля amount не найдены, ошибок нет
Отрицательные значения для поля annuity не найдены, ошибок нет
Отрицательные значения для поля contract_number не найдены, ошибок нет
Отрицательные значения для поля id_number не найдены, ошибок нет
Отрицательные значения для поля term_month не найдены, ошибок нет


Нарушения по числовым переменным также не обнаружены

Проверим значениях в полях дат:

In [33]:
data_type = 'date'
value_check(table, data_type)

Проверка обозначений по полю contract_date


,MAX(contract_date),MIN(contract_date),null_count
0,2018-12-30,2014-01-17,0


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 



In [34]:
date_value_update(table)

Среди дат по полю contract_date некорректные значения не найдены
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


В таблице контрактов некорректных значений в формате даты нет

### Пункт 3. Кодировка текстовых полей и создание словарей

In [35]:
def encoding_and_creating_dictionary(table):
    le = preprocessing.LabelEncoder()
    # создаём таблицу для словаря
    mycursor.execute("""CREATE TABLE {0}_dict (column_name VARCHAR(100), category VARCHAR(100), encoding INT)""".format(table))
    # кодируем все текстовые колонки
    mycursor.execute("""SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
    WHERE DATA_TYPE = 'varchar' AND TABLE_SCHEMA = '{0}'
    AND TABLE_NAME = '{1}'""".format(mydatabase, table))
    data_type_col = pd.DataFrame(mycursor.fetchall())[0].tolist()

    for i in range (0,len(data_type_col)):
        mycursor.execute("""SELECT id_number, {0} FROM {1}""".format(data_type_col[i], table))
        data = pd.DataFrame(mycursor.fetchall())
        data.columns = [x[0] for x in mycursor.description]
        data = data.dropna(how = 'any').reset_index(drop = True)
        data['encoded'] = le.fit_transform(data[data_type_col[i]])
        mycursor.execute("""ALTER TABLE {0} DROP {1}""".format(table, data_type_col[i]))
        mycursor.execute("""ALTER TABLE {0} ADD {1} INT""".format(table, data_type_col[i]))
        for j in range (0, len (data)):
            mycursor.execute("""UPDATE {0} SET {1} = {2} 
            WHERE {0}.id_number={3}""".format(table, data_type_col[i], data['encoded'][j], data['id_number'][j]))
            database.commit()
        
        # создаём для текущей текстовой колонки таблицу с данными для заполнения словаря
        to_dict = data[[data_type_col[i],'encoded']].drop_duplicates().sort_values(by = ['encoded']).reset_index(drop = True)
        to_dict.columns = ['category','encoding']
        to_dict['column_name'] = data_type_col[i]
        to_dict = to_dict[['column_name', 'category', 'encoding']]

        # заполняем подготовленными данными по текущему текстовому столбцу словарь кодировки
        sql = """INSERT INTO {0}_dict VALUES (%s, %s, %s)""".format(table)
        for k in range(0, to_dict.shape[0]):
            val = (to_dict.iloc[k][0], to_dict.iloc[k][1], int(to_dict.iloc[k][2]))
            mycursor.execute(sql, val)

            database.commit()

Перекодируем текстовые колонки в числа и создадим словари для каждой из таблиц:

In [36]:
tables = ['applications', 'contracts']
for i in tables:
    encoding_and_creating_dictionary(i)

Посмотрим на текущий вид таблиц:

In [37]:
for i in tables:
    mycursor.execute("SELECT * FROM {0}".format(i))

    data = pd.DataFrame(mycursor.fetchall())
    data.columns = [i[0] for i in mycursor.description]
    print("_ "*63)
    print()
    print('Таблица ', i)
    print("_ "*63)
    display(data.head())

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Таблица  applications
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


,id_number,issue_date,children,family,income,age_of_car,application_date,age,education,employed_by,gender,house_ownership,housing,income_type,marital_status,position
0,100003,2013-08-29,0.0,2.0,270000.0,NaN,2014-06-16,50,0.0,14,0,0.0,0.0,2.0,1.0,3.0
1,100004,2010-01-28,0.0,1.0,67500.0,26.0,2017-02-01,54,2.0,5,1,1.0,0.0,3.0,3.0,5.0
2,100006,2010-08-01,0.0,2.0,135000.0,NaN,2016-10-09,54,2.0,1,0,1.0,0.0,3.0,0.0,5.0
3,100007,2008-03-18,0.0,1.0,121500.0,NaN,2017-05-09,55,2.0,13,1,1.0,0.0,3.0,3.0,3.0
4,100008,2013-11-24,0.0,2.0,99000.0,NaN,2015-03-16,50,2.0,12,1,1.0,0.0,2.0,1.0,5.0


_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Таблица  contracts
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


,id_number,contract_number,amount,term_month,annuity,contract_date,type
0,100002,100875,406598,19,24701,2016-05-03,0
1,100003,100524,1293503,39,35699,2014-06-22,0
2,100004,100552,135000,23,6750,2017-08-01,1
3,100006,100493,312683,14,29687,2016-09-17,0
4,100007,100797,513000,26,21866,2017-06-09,0


Рассмотрим пример записей из словарей для полей education и gender из заявок и для поля type из контрактов

In [38]:
tables = ['applications', 'contracts']
fields = [['education','gender'],['type']]
for i in range(0, len(tables)):
    if tables[i] == 'applications':
        field_range = fields[0]
    if tables[i] == 'contracts':
        field_range = fields[1]
    for j in range(0, len(field_range)):
        mycursor.execute("SELECT * FROM {0}_dict WHERE column_name = '{1}'".format(tables[i], field_range[j]))          
        data = pd.DataFrame(mycursor.fetchall())
        data.columns = [x[0] for x in mycursor.description]
        print("_ "*63)
        print()
        print('Словарь для поля ', field_range[j],  " из таблицы ", tables[i])
        print("_ "*63)
        display(data)

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Словарь для поля  education  из таблицы  applications
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


,column_name,category,encoding
0,education,Higher education,0
1,education,Incomplete higher,1
2,education,Secondary / secondary special,2


_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Словарь для поля  gender  из таблицы  applications
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


,column_name,category,encoding
0,gender,Female,0
1,gender,Male,1


_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Словарь для поля  type  из таблицы  contracts
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


,column_name,category,encoding
0,type,Cash loans,0
1,type,Revolving loans,1


In [39]:
database.close()